# Using qualdocs to process Google Docs comments


This assumes you have `client_secret.json` in your working directory per the instructions in README.md

## Imports

In [2]:
import pandas as pd
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', 500)

In [3]:
import qualdocs

## Authentication

Authenticate with get_credentials(), which opens up a window in your browser to authenticate the first time you run it on a user account on this computer. It stores credentials to a .json file in `~/.credentials/`. Close the window and come back when authentication is complete.

In [4]:
credentials = qualdocs.get_credentials()

/home/staeiou/anaconda3/lib/python3.6/site-packages/oauth2client/_helpers.py:255: UserWarning: Cannot access /home/staeiou/.credentials/drive-python-quickstart.json: No such file or directory
  warnings.warn(_MISSING_FILE_MESSAGE.format(filename))



Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=LONGURL_TRUNCATED_FOR_PRIVACY

If your browser is on a different machine then exit and re-run this
application with the command-line parameter

  --noauth_local_webserver

Authentication successful.
Storing credentials to /home/staeiou/.credentials/drive-python-quickstart.json


In [5]:
service = qualdocs.get_service()

## Search for files to process

In [6]:
ids = qualdocs.get_file_ids(service, search="lorem")
ids

{'qualdocs test lorem 2': '1guJL7obwENn5GYOZarifxk3xPvWrk6NJ7WEfXQHTkBk',
 'qualdocs-test-lorem': '1EtYEx9U9KRfAOAh9LaSsmIQyDqiJ392qZJom1Jmv5MI'}

## Query and parse into pandas dataframe

In [7]:
json_dict = qualdocs.get_json_dict(service, ids)

In [8]:
df = qualdocs.json_to_df(json_dict)
df

text  \
code     subcode   sub_subcode  name                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         
topcode1                        qualdocs test lorem 2  Sed convallis purus lorem, ut euismod lorem scelerisque at. Vestibulum est diam, convallis ac dictum id, sodales nec nisl. Maecenas malesuada neque vel enim vestibulum laoreet.                                                                                                                                                                                                                                                                                                                                                                                                      
          subcode1              qualdocs-test-lorem    Nunc lacinia luctus mauris, vel malesuada lacus fermentum ullamcorper. Proin lacinia odio non tincidunt finibus.                                                                                                                                                                                                                                                                                                                                                                                                                                                                      
                                qualdocs-test-lorem    Vestibulum ac felis eget nisi iaculis condimentum. Morbi eros ligula, posuere id enim eu, dictum scelerisque magna.                                                                                                                                                                                                                                                                                                                                                                                                                                                                   
          subcode2              qualdocs-test-lorem    Lorem ipsum dolor sit amet, consectetur adipiscing elit. In enim sapien, fringilla vel sodales sed, tincidunt ut odio. Pellentesque habitant morbi tristique senectus et netus et malesuada fames ac turpis egestas.                                                                                                                                                                                                                                                                                                                                                                  
                                qualdocs-test-lorem    Duis a erat sodales, laoreet est vel, consequat libero. Donec nisl erat, venenatis sit amet lacinia sit amet, finibus in nulla.                                                                                                                                                                                                                                                                                                                                                                                                                                                       
                                qualdocs-test-lorem    Nunc lacinia luctus mauris, vel malesuada lacus fermentum ullamcorper. Proin lacinia odio non tincidunt finibus.                                                                                             

## Processing

Get list of codes

In [9]:
qualdocs.get_code_list(df)

['topcode1',
 'topcode1: subcode1',
 'topcode1: subcode1',
 'topcode1: subcode2',
 'topcode1: subcode2',
 'topcode1: subcode2',
 'topcode1: subcode3',
 'topcode1: subcode4',
 'topcode1: subcode4',
 'topcode2',
 'topcode2: subcode1',
 'topcode2: subcode1',
 'topcode2: subcode2: subsubcode1',
 'topcode2: subcode3',
 'topcode2: subcode3: subsubcode1',
 'topcode2: subcode3: subsubcode3',
 'topcode2: subcode3: subsubcode4',
 'topcode2: subcode4',
 'topcode3']

Get all top-level codes:

In [11]:
df.index.get_level_values(0).unique()

Index(['topcode1', 'topcode2', 'topcode3'], dtype='object', name='code')

Get all subcodes:

In [12]:
df.index.get_level_values(1).unique()

Index(['', ' subcode1', ' subcode2', ' subcode3', ' subcode4'], dtype='object', name='subcode')

Get all sub-subcodes:

In [13]:
df.index.get_level_values(2).unique()

Index(['', ' subsubcode1', ' subsubcode3', ' subsubcode4'], dtype='object', name='sub_subcode')

Filter by a top-level code:

In [14]:
df.xs('topcode1', level='code', drop_level=False)

text  \
code     subcode   sub_subcode name                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         
topcode1                       qualdocs test lorem 2  Sed convallis purus lorem, ut euismod lorem scelerisque at. Vestibulum est diam, convallis ac dictum id, sodales nec nisl. Maecenas malesuada neque vel enim vestibulum laoreet.                                                                                                                                                                                                                                                                                                                                                                                                      
          subcode1             qualdocs-test-lorem    Nunc lacinia luctus mauris, vel malesuada lacus fermentum ullamcorper. Proin lacinia odio non tincidunt finibus.                                                                                                                                                                                                                                                                                                                                                                                                                                                                      
                               qualdocs-test-lorem    Vestibulum ac felis eget nisi iaculis condimentum. Morbi eros ligula, posuere id enim eu, dictum scelerisque magna.                                                                                                                                                                                                                                                                                                                                                                                                                                                                   
          subcode2             qualdocs-test-lorem    Lorem ipsum dolor sit amet, consectetur adipiscing elit. In enim sapien, fringilla vel sodales sed, tincidunt ut odio. Pellentesque habitant morbi tristique senectus et netus et malesuada fames ac turpis egestas.                                                                                                                                                                                                                                                                                                                                                                  
                               qualdocs-test-lorem    Duis a erat sodales, laoreet est vel, consequat libero. Donec nisl erat, venenatis sit amet lacinia sit amet, finibus in nulla.                                                                                                                                                                                                                                                                                                                                                                                                                                                       
                               qualdocs-test-lorem    Nunc lacinia luctus mauris, vel malesuada lacus fermentum ullamcorper. Proin lacinia odio non tincidunt finibus.                                                                                                    